In [1]:
import flask
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlalchemy

In [ ]:
# serve.py

from flask import Flask, redirect, url_for, request
from flask import render_template

engine = create_engine("sqlite:///project.db")
# creates a Flask application, named app
app = Flask(__name__)
counter=0

# a route where we will display a welcome message via an HTML template
@app.route("/")
def init():
    return render_template('table.html')

@app.route('/search')
def search():
    tableNames=engine.table_names()
    return render_template('search.html',tableNames=tableNames, message="Please Select Table", hidden2="hidden",hidden3="hidden",hiddenDelete="hidden")
@app.route('/insert')
def insert():
    tableNames=engine.table_names()
    return render_template('insert.html',tableNames=tableNames, message="Please Select Table", hidden2="hidden",hidden3="hidden",hiddenDelete="hidden")

    
@app.route('/predefined', methods = ['POST', 'GET'])
def select():
    if request.method == 'POST':
        x = request.form["query_nr"]
        return redirect(url_for('table', x=x))
    
@app.route('/searchBox', methods = ['POST', 'GET'])
def getInput():
    if request.method == 'POST':
        first = request.form["first"]
        second = request.form["second"]
        return redirect(url_for('searchResult', first=first,second=second))
@app.route('/insertForm', methods = ['POST', 'GET'])
def insertForm():
    global counter
    global first
    global tableColumns
    if request.method == 'POST':
        counter=counter+1
        if counter==1:
            first = request.form["tableInsert"]
            message="Fill in the values of the columns "+ first
            query="select * from "+first
            tableColumns=engine.execute(query).keys();
            
        if counter==2:
            query= "Insert into "+ first+ " Values ( ";
            for i in range(len(tableColumns)):
                query = query + " '"+request.form[tableColumns[i]]+"',"
            query = query[:-1]
            query=query +");"
            engine.execute(query) 
            counter=0
            query=""
            return render_template('insert.html', message="The values were inserted succesfuly",hidden1="hidden", hiddenSubmit="hidden", hidden2="hidden")    

        
        return render_template('insert.html', message=message, tableColumns=tableColumns,hidden1="hidden")    
@app.route('/testForm', methods = ['POST', 'GET'])
def testForm():
    global counter
    global first
    global second
    global third

    if request.method == 'POST':
        counter=counter+1
        if counter==1:
            first = request.form["testDD"]
            message="Please select one columns of the table "+ first
            query="select * from "+first
            columns=engine.execute(query).keys();
            return render_template('search.html',tableColumns=columns, hidden1="hidden",hidden3="hidden",hiddenDelete="hidden", message=message)
        if counter == 2:
            second = request.form["testDD2"]
            message="Please input the value of the column "+ second
            return render_template('search.html', hidden1="hidden",hidden2="hidden",hiddenDelete="hidden", message=message)
        if counter == 3:
            third=request.form["value"]
            counter=0
            return redirect(url_for('searchResult', first=first,second=second, third=third))

        
        return render_template('search.html', message=message)
@app.route('/delete', methods = ['POST', 'GET'])
def delete():
    if request.method == 'POST':
        query="Delete From "+first+" where "+second+" = "+"'"+third+"'"
        test = engine.execute(query)
        return render_template('search.html', hidden1="hidden",hidden2="hidden",hidden3="hidden",hiddenSubmit="hidden",hiddenDelete="hidden", message="Row(s) deleted")
    
@app.route("/searchResult/<first>/<second>/<third>")
def searchResult(first,second, third):
    query="Select * From "+first+" where "+second+" = "+"'"+third+"'"
    test = engine.execute(query).fetchall();
    columns=engine.execute(query).keys();
    count=len(engine.execute(query).keys());
    return render_template('search.html', test=test, columns=columns, count= count,hidden1="hidden",hidden3="hidden",hidden2="hidden",hiddenSubmit="hidden")
    

@app.route("/table/<x>")
def table(x):
    list_of_predefined = ["",
                          "SELECT AVG(P.price)FROM Listing L, Pricing P WHERE L.id=P.id AND L.beds=8 ;", 
                          "SELECT AVG(S.review_scores_cleanliness) FROM Listing L, Amenities A, SCORE S, PROVIDES P WHERE L.id=S.id AND P.id=L.id AND P.amenities='tv' ;",
                         "SELECT DISTINCT L.host_id FROM Listing L, AVAILABLE_AT AV WHERE AV.id=L.id AND AV.available='t' AND AV.date <= '2019-09-31' AND AV.date >= '2019-03-00' ;",
                         "SELECT COUNT(*) FROM Host H1 WHERE EXISTS (SELECT L.id  FROM Host H2, Listing L WHERE H1.host_id < H2.host_id AND H1.host_name = H2.host_name AND L.host_id = H1.host_id );",
                         "SELECT DISTINCT AV.date FROM Listing L, AVAILABLE_AT AV, Host H WHERE L.host_id= H.host_id AND H.host_name='Viajes Eco' AND AV.id= L.id AND AV.available='t' ;",
                         "SELECT DISTINCT L.host_id, H.host_name FROM Listing L, Host H WHERE L.host_id = H.host_id GROUP BY L.host_id HAVING COUNT (*) = 1 ;",
                         "SELECT r1-r2 From (Select AVG(Pr2.price) As r2 FROM Listing L2, PROVIDES P2, Pricing Pr2 WHERE L2.id=P2.id AND P2.amenities<>'wifi' AND Pr2.id= L2.id), (select AVG(Pr1.price) as r1 FROM Listing L1, PROVIDES P1, Pricing Pr1  WHERE L1.id=P1.id AND P1.amenities='wifi' AND Pr1.id= L1.id);",
                         "Select s1-s2 FROM(SELECT AVG(Pr1.price) AS s1 FROM Listing L1, Pricing Pr1 WHERE L1.city='Berlin' AND Pr1.id=L1.id AND L1.beds=8 ), (SELECT AVG(Pr2.price) AS s2 FROM Listing L2, Pricing Pr2 WHERE L2.city='Madrid' AND Pr2.id=L2.id AND L2.beds=8);",
                         "SELECT DISTINCT L.host_id, H.host_name FROM Listing L, Host H WHERE L.country_code='ES' AND H.host_id = L.host_id ORDER BY (SELECT COUNT(*) FROM Listing L1 WHERE L.host_id = L1.host_id AND L.id < L1.id) DESC LIMIT 10;",
                         "SELECT L.id,L.name FROM Listing L, Score S WHERE L.city='Barcelona' AND S.id=L.id  ORDER BY (S.review_scores_cleanliness) DESC LIMIT 10;",
                         "SELECT COUNT(DISTINCT  L.host_id), N.city FROM Neighbourhood N, Listing L, Described_Description D WHERE L.id = D.id AND L.neighbourhood = N.neighbourhood AND D.square_feet <> -1 GROUP BY N.city ORDER BY N.city ASC;",
                         "WITH RankedTable AS(  SELECT neighbourhood, review_scores_rating,  ROW_NUMBER() OVER (PARTITION BY neighbourhood ORDER BY review_scores_rating) AS rank, COUNT(*) OVER (PARTITION BY neighbourhood) AS number_total FROM (SELECT L.neighbourhood, S.review_scores_rating FROM Listing L, Score S WHERE L.id=S.id AND L.city = 'Madrid') ) SELECT R.neighbourhood FROM RankedTable R WHERE R.rank = R.number_total / 2 + 1 ORDER BY R.review_scores_rating DESC LIMIT 5;",
                         "SELECT L.host_id, H.host_name FROM Listing L, Host H WHERE L.host_id = H.host_id GROUP BY L.host_id HAVING COUNT(*) = (SELECT COUNT(L.id) as max_listings FROM Listing L GROUP BY host_id ORDER BY COUNT (L.id) DESC limit 1)",
                         "SELECT AVG(AV.price) FROM Listing L, Available_at AV, Score S, Verified_by V WHERE L.city='Berlin' AND L.beds >= 2 AND S.review_scores_location >= 8 AND S.id= L.id 	  AND L.cancellation_policy = 'flexible' AND L.host_id = V.host_id 	  AND (V.host_verifications='government_id' OR V.host_verifications='offline_government_id') 	  AND L.id=AV.id AND AV.date >= '2019-03-01' AND AV.date <= '2019-04-30' AND AV.available = 't';",
                         "SELECT * from( SELECT L.id, L.accommodates, ROW_NUMBER() OVER( 	PARTITION BY L.accommodates 	ORDER BY (SELECT S.review_scores_rating) ) rank_for_this_type FROM Listing L, Score S WHERE L.id = S.id and S.review_scores_rating >= 0 AND 2 >= (SELECT COUNT (*) FROM Provides P WHERE P.id=L.id AND(P.amenities='Wifi' OR P.amenities='Internet'     OR P.amenities='TV'OR P.amenities='Free street parking')) ) WHERE rank_for_this_type <= 5;",
                         "SELECT * FROM(     SELECT  L.host_id, H.host_name, L.id, L.name, ROW_NUMBER() OVER (         PARTITION BY L.host_id         ORDER BY (SELECT COUNT(R.review_id))     )rank     FROM Listing L, Reviewed R, Host H     WHERE R.id=L.id AND H.host_id = L.host_id    GROUP BY L.id ) WHERE rank <= 3",
                         "SELECT nc.neighbourhood, cc.city, amenities FROM ( 	SELECT  ROW_NUMBER () OVER (        PARTITION BY  n.neighbourhood         ORDER BY count(lp.id) DESC     )AS c,  	n.neighbourhood, lp.amenities, count(*)  	FROM Provides lp JOIN Listing l ON l.ID = lp.id 	JOIN NEIGHBOURHOOD n ON n.neighbourhood = l.neighbourhood 	WHERE n.city IN 	(SELECT City.city FROM City WHERE city = 'Berlin') and l.room_type = 'Private room' 	GROUP BY  n.neighbourhood, lp.amenities 	ORDER BY count(lp.id) DESC )a JOIN NEIGHBOURHOOD nc on a.neighbourhood = nc.neighbourhood JOIN CITY cc ON cc.city = nc.city WHERE a.c < 4 ORDER BY nc.neighbourhood, amenities DESC;",
                         "SELECT AV2-AV1 FROM(   (SELECT H.host_id AS ID1, COUNT(V.host_verifications) AS N1, AVG(S.review_scores_communication) AS AV1   FROM Verified_by V, Host H,Listing L, SCORE S   WHERE V.host_id = H.host_id and L.host_id=h.host_id AND S.id = L.id   GROUP BY H.host_id   ORDER BY COUNT(host_verifications) DESC LIMIT 1),    (SELECT H2.host_id AS ID2, COUNT(V2.host_verifications) AS N2, AVG(S2.review_scores_communication) AS AV2   FROM Verified_by V2, Host H2, Listing L2, SCORE S2   WHERE V2.host_id = H2.host_id and L2.host_id=h2.host_id AND S2.id = L2.id   AND S2.review_scores_communication != -1    GROUP BY H2.host_id ORDER BY COUNT(host_verifications) ASC LIMIT 1));",
                         "select city From (     Select count(r.id) as reviews ,listing.city as city     From Listing listing, Reviewed r,         (        Select rt         From (         Select avg(l.accommodates)as avg, l.room_type as rt From Listing l GROUP BY l.room_type)         where avg>3)     where r.id=listing.id and listing.room_type=rt     group by listing.city) order by reviews desc limit 1;",
                         "Select * From(     select count(distinct a.id) as occupiedNr, l1.neighbourhood as occupiedN     from available_at a, host h, Listing l1     where L1.id in (       SELECT  L3.id as occupiedId       FROM Listing L3, available_at A3       WHERE L3.id = A3.id AND A3.date >= '2019-01-01' and A3.date <= '2019-12-31'       GROUP BY L3.id       HAVING COUNT(A3.available = 'f') >= 1     ) and h.host_id= l1.host_id and a.id=l1.id and h.host_since < '2017-06-01' and l1.city='Madrid'     group by l1.neighbourhood),      (Select count( distinct l2.id) as totalNr, l2.neighbourhood as N     from Listing l2     where l2.city='Madrid'     group by l2.neighbourhood) where occupiedN=N and occupiedNr >= totalNr / 2;",
                         "SELECT CC1 FROM (select L.country_code AS CC1, count(DISTINCT L.id) AS CA From  Listing L Where L.id in(   select A.id   from  available_at A   where  A.date >= '2018-01-01' AND A.date <= '2018-12-31'    GROUP BY A.id     HAVING COUNT(A.available = 't') >= 1 ) group by L.country_code),  (select L1.country_code AS CC2, count(DISTINCT L1.id) AS CTOT From  Listing L1 Where L1.id in(   select A1.id   from  available_at A1 where  A1.date >= '2018-01-01' AND A1.date <= '2018-12-31'    GROUP BY A1.id ) group by L1.country_code) WHERE CC1 = CC2 AND CA >= CTOT / 5",
                         "SELECT NE1 FROM (SELECT L.neighbourhood AS NE1, COUNT(DISTINCT L.id) AS COUNT1 FROM Neighbourhood N, Listing L WHERE L.cancellation_policy = 'strict_14_with_grace_period' AND N.city = 'Barcelona' AND L.neighbourhood = N.neighbourhood GROUP BY L.neighbourhood),  (SELECT L1.neighbourhood AS NE2, COUNT(DISTINCT L1.id) AS COUNT2 FROM Neighbourhood N1, Listing L1 WHERE N1.city = 'Barcelona' AND L1.neighbourhood = N1.neighbourhood GROUP BY L1.neighbourhood) WHERE NE1 = NE2 AND COUNT1 > COUNT2 / 20"] 
    test = engine.execute(list_of_predefined[int(x)]).fetchall();
    columns=engine.execute(list_of_predefined[int(x)]).keys();
    count=len(engine.execute(list_of_predefined[int(x)]).keys());
    return render_template('table.html', test=test, columns=columns, count= count)
# run the application
if __name__=="__main__":
     app.run(port=5000, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/May/2019 21:36:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2019 21:36:30] "POST /predefined HTTP/1.1" 302 -
127.0.0.1 - - [31/May/2019 21:36:31] "GET /table/22 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2019 21:36:38] "POST /predefined HTTP/1.1" 302 -
127.0.0.1 - - [31/May/2019 21:37:23] "GET /table/21 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2019 21:37:31] "POST /predefined HTTP/1.1" 302 -
127.0.0.1 - - [31/May/2019 21:37:32] "GET /table/1 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2019 21:37:35] "POST /predefined HTTP/1.1" 302 -
127.0.0.1 - - [31/May/2019 21:37:36] "GET /table/4 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2019 21:37:38] "POST /predefined HTTP/1.1" 302 -
127.0.0.1 - - [31/May/2019 21:37:40] "GET /table/8 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2019 21:37:43] "POST /predefined HTTP/1.1" 302 -
127.0.0.1 - - [31/May/2019 21:37:48] "GET /table/7 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2019 21:37:51] "POST /predef